<a href="https://colab.research.google.com/github/IgorZorkov/number-plates-ocr/blob/main/number_plates_ocr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install onnxruntime
!pip install onnx

In [ ]:
%cd /content

In [ ]:
import gdown
url = f"https://drive.google.com/open?id=1MmrEB8f91ZmS6k_vbQ8C5VIXPthVd4WY"
#https://drive.google.com/open?id=1MmrEB8f91ZmS6k_vbQ8C5VIXPthVd4WY
output = 'number_plates_ocr.zip'
gdown.download(url, output, quiet=False)

In [ ]:
from zipfile import ZipFile
with ZipFile('/content/number_plates_ocr.zip', 'r') as zipObj:
   zipObj.extractall('/content')

In [ ]:
import cv2
import numpy as np
import onnxruntime
import json
import os
import statistics
from scipy.special import softmax
import time

def read_vocab(path):
    with open(path) as f:
        vocab = json.load(f)
    return vocab

def norm(x):
    mean = [0.5, 0.5, 0.5]
    std = [0.5, 0.5, 0.5]
    x = x / 255.0
    x[0, :, :] -= mean[0]
    x[1, :, :] -= mean[1]
    x[2, :, :] -= mean[2]
    x[0, :, :] /= std[0]
    x[1, :, :] /= std[1]
    x[2, :, :] /= std[2]
    return x

def decode_text(tokens, vocab, vocab_inp):
    start = vocab.get('<s>')
    end = vocab.get('</s>')
    unk = vocab.get('<unk>')
    pad = vocab.get('<pad>')
    text = ''
    for i in tokens:
        if i == end:
            break
        if i not in [end, start, pad, unk]:
            text += vocab_inp[i]
    return text

class OnnxEncoder(object):
    def __init__(self, model_path):
        self.model = onnxruntime.InferenceSession(model_path, providers=onnxruntime.get_available_providers())

    def __call__(self, image):
        onnx_inputs = {self.model.get_inputs()[0].name: np.asarray(image, dtype='float32')}
        onnx_output = self.model.run(None, onnx_inputs)[0]

        return onnx_output

class OnnxDecoder(object):
    def __init__(self, model_path):
        self.model = onnxruntime.InferenceSession(model_path, providers=onnxruntime.get_available_providers())
        self.input_names = {input_key.name: idx for idx, input_key in enumerate(self.model.get_inputs())}

    def __call__(self, input_ids,
                 encoder_hidden_states):
        onnx_inputs = {"input_ids": input_ids, "encoder_hidden_states": encoder_hidden_states}
        onnx_output = self.model.run(['logits'], onnx_inputs)
        return onnx_output

class OnnxEncoderDecoder(object):
    def __init__(self, model_path):
        self.encoder = OnnxEncoder(os.path.join(model_path, "encoder_model.onnx"))
        self.decoder = OnnxDecoder(os.path.join(model_path, "decoder_model.onnx"))
        self.vocab = read_vocab(os.path.join(model_path, "vocab.json"))
        self.vocab_inp = {self.vocab[key]: key for key in self.vocab}
        self.threshold = 0.5
        self.max_len = 64

    def run(self, image):
        image = cv2.resize(image, (384, 384), 1)

        pixel_values = cv2.split(np.array(image))
        pixel_values = norm(np.array(pixel_values))
        pixel_values = np.array([pixel_values])
        encoder_output = self.encoder(pixel_values)
        ids = [self.vocab["<s>"], ]
        mask = [1, ]
        scores = []
        for i in range(self.max_len):
            input_ids = np.array([ids])
            attention_mask = np.array([mask])
            decoder_output = self.decoder(input_ids=input_ids, encoder_hidden_states=encoder_output)
            pred = decoder_output[0][0]
            pred = softmax(pred, axis=1)
            max_index = pred.argmax(axis=1)
            if max_index[-1] == self.vocab["</s>"]:
                break
            scores.append(pred[max_index.shape[0] - 1, max_index[-1]])
            ids.append(max_index[-1])
            mask.append(1)
        print("score：{}".format(statistics.mean(scores)))
        if self.threshold < statistics.mean(scores):
          text = decode_text(ids, self.vocab, self.vocab_inp)
        else:
          text = ""
        return text

model = OnnxEncoderDecoder("/content")

In [ ]:
image = cv2.imread("/content/test.jpg")
image = image[..., ::-1]
result = model.run(image)
print("text: ", result)